In [ ]:
# !pip install tensor2tensor[tensorflow_gpu]
# !pip install pandas
# !pip install sklearn
# !pip install nltk
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import tensorflow as tf
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

tfe = tf.contrib.eager
tfe.enable_eager_execution()
Modes = tf.estimator.ModeKeys

### Register Problem

In [1]:
from tensor2tensor.data_generators import semantic_search
search_problem = semantic_search.SemanticSearch()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## Generating Data

In [ ]:
search_problem.generate_data(data_dir='/tf/tensor2tensor/t2t_data', tmp_dir='/tf/tensor2tensor/datagen')

### Building Model and Train

In [2]:
PARAMS = {}
PARAMS['T2T_Problem'] = 'semantic_search'
PARAMS['T2T_Model'] = 'transformer'
PARAMS['T2T_HPARAMS'] = 'transformer_base'
PARAMS['train_steps'] = 100000
PARAMS['eval_steps'] = 100
PARAMS['keep_checkpoint_max'] = 3

In [3]:
import tensorflow as tf
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment, create_hparams
from tensor2tensor.utils import registry
from tensor2tensor import models, problems

hparams = create_hparams(PARAMS['T2T_HPARAMS'])

FLAGS = tf.flags
FLAGS.problems = PARAMS['T2T_Problem']
FLAGS.model = PARAMS['T2T_Model']
FLAGS.schedule = "train_and_evaluate"

In [6]:
PARAMS['TMP_DIR'] = '/tf/datagen/'
PARAMS['DATA_DIR'] = '/tf/t2t_data'
PARAMS['TRAIN_DIR'] = '/tf/t2t_train' 
PARAMS['OUTPUT_DIR'] = 'tf/t2t_train/semantic_search'

In [ ]:
RUN_CONFIG = create_run_config(hparams, model_dir=PARAMS['TRAIN_DIR'])

In [ ]:
exp_fn = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=PARAMS['T2T_Model'],
        problem_name=PARAMS['T2T_Problem'],
        data_dir=PARAMS['DATA_DIR'],
        train_steps=PARAMS['train_steps'],
        eval_steps=PARAMS['eval_steps']
    )
exp_fn.train_and_evaluate()

## Getting latest checkpoint

In [ ]:
import os

ckpt_path = tf.train.latest_checkpoint(PARAMS['TRAIN_DIR'])
ckpt_path 

### Getting translatte model

In [ ]:
hparams_eval = trainer_lib.create_hparams(PARAMS['T2T_HPARAMS'], data_dir=PARAMS['DATA_DIR'], 
                                     problem_name=PARAMS['T2T_Problem'])

translate_model = registry.model(PARAMS['T2T_Model'])(hparams_eval, Modes.EVAL)

encoders = problems.problem(PARAMS['T2T_Problem']).feature_encoders(PARAMS['DATA_DIR'])

def encode(input_str, output_str=None):
    """Input str to features dict, ready for inference"""
    inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
    return {"inputs": batch_inputs}

def decode(integers):
    """List of ints to str"""
    integers = list(np.squeeze(integers))
    if 1 in integers:
        integers = integers[:integers.index(1)]
    return encoders["inputs"].decode(np.squeeze(integers))

# Restore and translate!
def translate(inputs):
    encoded_inputs = encode(inputs)
    with tfe.restore_variables_on_create(ckpt_path):
        model_output = translate_model.infer(encoded_inputs)["outputs"]
        return decode(model_output)

In [ ]:
conala_df = pd.read_json("/tf/datagen/conala-train.json")
conala_df.head()

In [ ]:
intent, code = conala_df.iloc[4].intent, conala_df.iloc[4].snippet
print(intent)
print(code)
translate(code)